# 1. Initialize notebook

## Import libraries

In [ ]:
from data_collections import *
from image_analysis import *
from event_detection import *

## Load analysis parameters

In [ ]:
with open('parameters.json', 'r') as f:
    parameters = json.load(f)

## Set global constants

In [ ]:
# Set top data directory
top_dir = ''

# Save data channels
channels = parameters['channels']

# 2. Load experiments

## Log experiments to database

In [ ]:
# Load and save metadata for all experiments
log_movies(top_dir, parameters['channel_aliases'], overwrite=parameters['overwrite'])

# Preview current database
preview_database()

## Set data to analyze

In [ ]:
# Retrieve metadata for all movies in database
metadata = load_metadata()

## Preview sample movie

In [ ]:
# Load single movie
movie = next(iter_movies(metadata))
movie.deskewed

In [ ]:
# View image stack of specified channel
movie.deskewed[channels[0]]

In [ ]:
# View image stack of specified frame and channel
movie.deskewed[channels[0]].sel(t=6)

# 3. Run analysis pipeline

## Process all images

In [ ]:
# Set list of movies to analyze
movies = list()
for _, row in metadata.iterrows():
    args = row[['experiment', 'movie']]
    movie = MultiChannelMovie.load(*args)
    movies.append(movie)

## Filter images

In [ ]:
# Filter the deskewed images and save as processed
args = filter_image, 'deskewed', 'processed'
kwargs = parameters['filter_image']

apply_movies(movies, 'map_blocks', *args, **kwargs)

## Locate blobs

In [ ]:
# Locate blobs in filtered images
args = [locate_blobs, 'processed', None, 'blobs']
kwargs = parameters['locate_blobs']

apply_movies(movies, 'compute', *args, **kwargs)

In [ ]:
# Visualize the results for all frames
movie_index = 0
movie = movies[movie_index]
frames = movie.get_frames()
movie.view_labels(use_filters=False, frames=frames)

## Calculate background

In [ ]:
for movie in tqdm(movies, total=len(movies)):
    try:
        # Generate cell mask based on min/mean projections across movie
        print(f"Calculating background of {str(movie)}...")
        
        bgd_images = get_bgd_images(movie, **parameters['get_bgd_images'])
        save_tif(movie, 'Background', bgd_images)
        
        cell_masks = {channel : get_cell_mask(bgd_image, **parameters['get_cell_mask']) for channel, bgd_image in bgd_images.items()}
        save_tif(movie, 'CellMask', cell_masks)
    except Exception as e:
        print(f"Error: {e}.")

## Calculate intensities

In [ ]:
# Calculate blob intensities in filtered image
param_dict = parameters['extract_intensities']

for movie in tqdm(movies, total=len(movies)):
    try:
        # Configure channels from data
        channels = movie.get_channels('deskewed')
        frames = movie.get_frames('deskewed')
        param_dict['labeled_dir'] = movie.metadata['paths']['labeled']
        
        for channel in channels:
            tif_dir = os.path.join(param_dict['labeled_dir'], channel)
            if overwrite and os.path.exists(tif_dir): shutil.rmtree(tif_dir)
            if not os.path.exists(tif_dir): os.makedirs(tif_dir)
        
        # Set image metadata (mask from deskewed image is constant throughout)
        skew_mask = movie._movies['deskewed'][channels[0]].sel(t=frames[0]).data.compute() > 0
        
        # Generate cell mask based on min/mean projections across movie
        bgd_images = load_tif(movie, 'Background')
        cell_masks = load_tif(movie, 'CellMask')
        param_dict['images'] = {'skew_mask': skew_mask, 'cell_masks': cell_masks, 'bgd_images': bgd_images}
        
        args = extract_intensities, 'deskewed', 'blobs', 'intensities'
        kwargs = {'overwrite': parameters['overwrite'], 'param_dict': param_dict}
        
        movie.compute(*args, **kwargs)
    except Exception as e:
        print(f"Error: {e}.")

## Filter particles

In [ ]:
movies = iter_movies(metadata)

data_type_out = 'Filters'

for movie in tqdm(movies, total=len(movies)):
    try:
        print(f"Calculating {data_type_out} of {str(movie)}...")
        
        # Cluster blobs into signal versus background
        dict_out = cluster_blobs(movie, **parameters['template_radii'])

        # Save final results as movie data
        movie._data[data_type_out] = dict_out

        # Export results to separate CSV for each channel
        for channel, df in dict_out.items():
            csv_path = os.path.join(movie.metadata['paths']['tracking'], 
                                    data_type_out.title() + '_' + channel + '.csv')
            
            df.to_csv(csv_path, index=False)

        # Set the data types attributes
        movie._data_types.add(data_type_out)

        # Save updated parameters
        movie.save()
    except Exception as e:
        print(f"Error: {e}.")

In [ ]:
# Visualize the results for all frames
movie_index = 0
movie = movies[movie_index]
frames = movie.get_frames()
movie.view_labels(use_filters=True, frames=frames)

## Track particles

In [ ]:
data_type_out = 'Tracked'
for movie in tqdm(movies, total=len(movies)):
    try:
        print(f"Calculating {data_type_out} of {str(movie)}...")
        
        # Calculate information for each pair of neighboring particles
        movie._data['filters'] = load_csv(movie, 'filters')
        link_tracks(movie, parameters['link_tracks'])

        # Set the data types attributes
        movie._data_types.add(data_type_out)

        # Save updated parameters
        movie.save()
    except Exception as e:
        print(f"Error: {e}.")

## Identify nearby particles

In [ ]:
data_type_out = 'Neighbors'
for movie in tqdm(movies, total=len(movies)):
    try:
        print(f"Calculating {data_type_out} of {str(movie)}...")
        
        # Calculate information for each pair of neighboring particles
        movie._data['filters'] = load_csv(movie, 'filters')
        
        # Remove untracked and duplicate objects
        movie._data['tracked'] = load_csv(movie, 'tracked')
        
        get_neighbors(movie, **parameters['get_neighbors'])

        # Set the data types attributes
        movie._data_types.add(data_type_out)

        # Save updated parameters
        movie.save()
    except Exception as e:
        print(f"Error: {e}.")

## Find collisions

In [ ]:
data_type_out = 'Collisions'
for movie in tqdm(movies, total=len(metadata)):
    try:
        print(f"Calculating {data_type_out} of {str(movie)}...")
        
        
        # Identify collisions
        find_collisions(movie, **parameters['find_collisions'])
        
        # Print summary of results
        num_collisions = len(movie._data['collisions'][channels[0]])
        num_frames = len(movie.get_frames())
        print(f"Found a total of {num_collisions} collisions ({num_collisions * 60 / (params['frame_rate'] * num_frames):0.1f} per minute, " + \
              f"{num_collisions / num_frames:0.1f} per frame), with an average of " + \
              f" and ".join([f"{int(movie._data['tracked'][channel].groupby('frame')['track'].count().mean())} {channel}s" for channel in channels]) + ' per frame')
        
        # Set the data types attributes
        movie._data_types.add(data_type_out)

        # Save updated parameters
        movie.save()
    except Exception as e:
        print(f"Error: {e}.")
        raise

## Find conversions

In [ ]:
data_type_out = 'Conversions'
for movie in movies:
    try:
        print(f"Calculating {data_type_out} of {str(movie)}...")
        
        # Identify conversions
        find_conversions(movie, **parameters['find_conversions'])
        
        # Print summary of results
        num_overlaps = len(movie._data['conversions'][channels[0]])
        num_conversions = sum(movie._data['conversions'][channels[0]]['conversion'])
        num_frames = len(movie.get_frames())
        print(f"Found a total of {num_overlaps} overlaps and {num_conversions} conversions " + \
              f"({num_conversions * 60 / (params['frame_rate'] * num_frames):0.1f} per minute, " + \
              f"{num_conversions / num_frames:0.1f} per frame), with an average of " + \
              f" and ".join([f"{int(movie._data['tracked'][channel].groupby('frame')['track'].count().mean())} {channel}s" for channel in channels]) + ' per frame')
        
        # Set the data types attributes
        movie._data_types.add(data_type_out)
        
        # Save updated parameters
        movie.save()
    except Exception as e:
        print(f"Error: {e}.")